# People Data Labs Exploration

In [1]:
import os
import requests
import pandas as pd
import numpy as np 
import boto3

bsdir = "s3://ipsos-dvd/"
data_dir = bsdir + "ppl/data/"
data_ppl = "s3://external-people-data-labs/"


from pyspark import SparkContext
from pyspark.ml.feature import StringIndexer


from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import (
    StructType,
    LongType,
    StructField,
    IntegerType,
    StringType,
    DoubleType,
    TimestampType,
    ArrayType
)
import pyspark.sql.functions as F
sc.install_pypi_package("thefuzz")
from thefuzz import process, fuzz

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1710789961131_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 184.7 MB/s eta 0:00:00


In [6]:
## check nepal for megan
nep = spark.read.option("recursiveFileLookup", "true").parquet("s3://external-veraset-data-us-west-2/nepal/*")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
nep.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

28284405

In [9]:
nep.agg(F.countDistinct("caid")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|count(caid)|
+-----------+
|     282536|
+-----------+

In [10]:
nep.agg(F.min("utc_timestamp"), F.max("utc_timestamp")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+
| min(utc_timestamp)|  max(utc_timestamp)|
+-------------------+--------------------+
|2024-01-01 00:00:00|2024-01-14 23:59:...|
+-------------------+--------------------+

### Pull Experience from API

In [ ]:
import requests
 
API_KEY = "FAgFuQO8.pa4ELHbKch2xGuBpLjDPZZsKC9BFcJDmxB1DAGEKAHvXRj5JoJt1xHpE"
PRODUCT_API_PATH = "https://app.deweydata.io/external-api/v3/products/8279ab69-568c-4b9a-b08e-d94a86525189/files"

results = requests.get(url=PRODUCT_API_PATH+"/metadata",
                       headers={
                        "X-API-KEY": API_KEY,
                        'accept': 'application/json'
                       })
print(results.json())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'total_files': 1024, 'total_size': 220265634728, 'partition_column': None, 'partition_aggregation': None, 'min_partition_key': None, 'max_partition_key': None}

In [ ]:
# import requests library to call API endpoint
import requests

s3 = boto3.resource('s3')

# loop through all API result pages, keeping track of number of downloaded files
page = 1
download_count = 0
while True:
    # get results from API endpoint, using API key for authentication, for a specific page
    results = requests.get(url=PRODUCT_API_PATH,
                           params={'page': page},
                           headers={'X-API-KEY': API_KEY,
                                    'accept': 'application/json'
                                   })
    response_json = results.json()

    # for each result page, loop through download links and save to your computer
    for link_data in response_json['download_links']:
        print(f"Downloading file {link_data['file_name']}...")
        data = requests.get(link_data['link'])
        object = s3.Object('ipsos-dvd', 'ppl/data/' + link_data['file_name'])
        object.put(Body=data.content)
        download_count += 1

    # only continue if there are more result pages to process
    total_pages = response_json['total_pages']
    if page >= total_pages:
        break
    page += 1

print(f"Successfully downloaded {download_count} files.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'download_links'
Traceback (most recent call last):
KeyError: 'download_links'



### Load Experience Data

In [7]:
exp =  spark.read.option("multiLine", "true") \
                .option("escape", "\"") \
                .option("quote", '"').csv(data_ppl + "people-experience/data/", header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
exp = exp.filter(F.col("COMPANY_LOCATION_COUNTRY") == "united states")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
exp.write.mode("overwrite").parquet(data_dir + "people-experience_us.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Reload

In [10]:
exp = spark.read.parquet(data_dir + "people-experience_us.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
print(exp.limit(20).toPandas()['COMPANY_WEBSITE'].to_string())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0                         None
1                 princess.com
2         asburypark.k12.nj.us
3               wellsfargo.com
4                   goarmy.com
5             sutterhealth.org
6             digitalriver.com
7             lfscarolinas.org
8                  fontana.org
9                      dhs.gov
10                vitalant.org
11                   uhaul.com
12                        None
13                  plygem.com
14                   aecom.com
15                   aecom.com
16                  subway.com
17                  scmlaw.com
18     riversidehealthcare.org
19    watermarkcommunities.com

In [12]:
tech = pd.read_csv(data_dir + "tech_companies_rto.csv")
tech['Company Name'] = tech['Company Name'].str.lower()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
names_to_get = tech['Company Name']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
firmnames = exp.select("COMPANY_NAME").distinct().collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
firmnames = [row.COMPANY_NAME for row in firmnames]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
firmnames[0:10]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['caesars entertainment, inc.', 'carnival cruise line', 'baylor college of medicine', 'gtt', 'turell group', "bruegger's bagels", 'xenex', 'lewis-clark state college', 'bytegraph creations', 'u.s. silica company']

In [17]:
names = pd.DataFrame({'tech' : tech['Company Name'].str.lower()})


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
names['matched'] = names.apply(lambda x: process.extractOne(x.tech, firmnames), axis=1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
names_to_get = [i[0] for i in names['matched']]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
websites = exp.filter(F.col("COMPANY_NAME").isin(list(names_to_get))).select("COMPANY_WEBSITE").distinct().collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
websites = [row.COMPANY_WEBSITE for row in websites]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
## get scoop flex data
flex = pd.read_csv(data_dir + "companies.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
agg = pd.read_csv(data_dir + "flex_index_aggregate.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
flex = flex.merge(agg.rename(columns={'company_name' : 'name'}), on = "name", how="left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
tech = tech.rename(columns={'Company Name' : 'name', 'RTO_1' : 'RTO_date_manual'})[['name', 'RTO_date_manual']]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
names['COMPANY_NAME'] = names['matched'].apply(lambda x: x[0])
names = names.rename(columns = {'tech' : 'name'})[['COMPANY_NAME', 'name']]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
tech = tech.merge(names, on = "name", how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
tech.loc[tech.RTO_date_manual.isna(), 'RTO_date_manual'] = "never"
tech = tech[['COMPANY_NAME', 'RTO_date_manual']]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
flexdomains = list(flex['domain'].str.lower())
flexnames = list(flex['name'].str.lower())
flex = spark.createDataFrame(flex[["name", "domain", "office_requirements", "requirement_details"]]).write.mode("overwrite").parquet(data_dir + "flex.parquet")
tech = spark.createDataFrame(tech).write.mode("overwrite").parquet(data_dir + "tech.parquet")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
(exp.filter(F.col("COMPANY_WEBSITE").isin(flexdomains) | 
           F.col("COMPANY_NAME").isin(flexnames) |
          F.col("COMPANY_NAME").isin(names_to_get) | 
           F.col("COMPANY_WEBSITE").isin(websites)
          )
#     .join(flex, 
#           on = (flex.name == exp.COMPANY_NAME) | (flex.domain == exp.COMPANY_WEBSITE))
#     .join(tech, on = "COMPANY_NAME")
).write.mode("overwrite").parquet(data_dir + "people-experience_us_tech.parquet")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### TODO: restart from here feb 4 2024

In [31]:
exp = spark.read.parquet(data_dir + "people-experience_us_tech.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Merge in flex policy info

In [32]:
flex = spark.read.parquet(data_dir + "flex.parquet")
exp = exp.join(flex, on = (flex.name == exp.COMPANY_NAME) | (flex.domain == exp.COMPANY_WEBSITE),
         how = "left")



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
tech = spark.read.parquet(data_dir + "tech.parquet")
exp = exp.join(tech, on = "COMPANY_NAME", how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Merge in people info

In [34]:
people = spark.read.csv(data_ppl + "people-core", header=True)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
exp = exp.join(people, on = "ID", how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
exp.filter(F.col("LOCATION_COUNTRY") == # filter on location again to 
           "united states").write.mode("overwrite").parquet(data_dir + "people-experience_us_tech_people.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Construct individual tenure profile by company pre and post apple RTO

In [12]:
exp = spark.read.parquet(data_dir + "people-experience_us_tech_people.parquet")

# rto was on april 11, 2022. Let's restrict to people with END_DATE 4 months before that or later
exp = exp.filter(exp.START_DATE.isNotNull() & ((exp.END_DATE >= "2022-01-01") | exp.END_DATE.isNull()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
from datetime import datetime, timedelta

start_date = datetime(2021, 1, 1)
end_date = datetime(2024, 1, 1)

# Calculate the total number of months in the range
total_months = (end_date.year - start_date.year) * 12 + end_date.month - start_date.month

# Generate the list of dates
date_list = [start_date + timedelta(days=31*i) for i in range(total_months)]
# Adjust each date to the first of the month
date_list = [datetime(date.year, date.month, 1) for date in date_list]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:

# Duplicate and append DataFrames
all_dfs = []
for date in date_list:
    # Add a date column to the DataFrame
    df_with_date = exp.withColumn("month", F.lit(date.strftime("%Y-%m-%d")))
    all_dfs.append(df_with_date)

# Concatenate all DataFrames
final_df = all_dfs[0]
for df in all_dfs[1:]:
    final_df = final_df.union(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
final_df = final_df.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# explode TITLE_LEVELS column vertically
final_df = final_df.withColumn("TITLE_LEVELS", F.explode(F.split("TITLE_LEVELS", ",")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
## calculate title
# Inverted mapping of title levels to their categorical ranks
title_ranking_inverted = {
    "cxo": 10,
    "vp": 9,
    "director": 8,
    "partner": 7,
    "owner": 6,
    "senior": 5,
    "manager": 4,
    "entry": 3,
    "training": 2,
    "unpaid": 1
}

# Creating a new column 'title_categorical' with the inverted ranks
rank_expr_inverted = "case "
for title, rank in title_ranking_inverted.items():
    rank_expr_inverted += f"when TITLE_LEVELS = '{title}' then {rank} "
rank_expr_inverted += "else null end"

final_df = final_df.withColumn("title_categorical", F.expr(rank_expr_inverted))

# now only keep observations with highest title_categorical
windowSpec = Window.partitionBy('ID', 'month', "COMPANY_NAME").orderBy(F.col('title_categorical').desc())

# Add row numbers to each row within each partition
df_with_row_numbers = final_df.withColumn('row_number', F.row_number().over(windowSpec))

# Filter to keep only the rows with the largest value in the varying column for each group
df_filtered = df_with_row_numbers.filter(F.col('row_number') == 1).drop('row_number')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# calculate tenure variable
final_df = (final_df.filter( ((F.col("END_DATE") >= F.col("month")) | F.col("END_DATE").isNull()) & 
                            (F.col("START_DATE") <= F.col("month")))
        .withColumn("tenure", F.datediff(F.col("month"), F.col("START_DATE"))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
reference_date = F.to_date(F.lit(final_df.agg(F.min(F.col("month"))).collect()[0][0]))
reference_date

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Column<'to_date(2021-01-01)'>

In [20]:
# generate t_col variable
final_df = final_df.withColumn("time_col", (F.year(F.col("month")) - F.year(reference_date)) * 12 + 
                   (F.month(F.col("month")) - F.month(reference_date)) + 1)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Create a StringIndexer
indexer = StringIndexer(inputCol="COMPANY_NAME", outputCol="id_col")

# Fit the indexer to the DataFrame and transform the DataFrame
final_df = indexer.fit(final_df).transform(final_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
final_df = final_df.withColumn("id_col", F.col("id_col").cast(IntegerType())) # generate id_col variable

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
final_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID', 'COMPANY_NAME', 'COMPANY_RAW', 'COMPANY_ID', 'COMPANY_FOUNDED', 'COMPANY_TWITTER_URL', 'COMPANY_LOCATION_POSTAL_CODE', 'COMPANY_LOCATION_REGION', 'COMPANY_LOCATION_GEO', 'TITLE_NAME', 'TITLE_LEVELS', 'COMPANY_SIZE', 'COMPANY_LOCATION_METRO', 'COMPANY_LOCATION_ADDRESS_LINE_2', 'TITLE_ROLE', 'TITLE_SUB_ROLE', 'COMPANY_LOCATION_NAME', 'START_DATE', 'TITLE_RAW', 'COMPANY_LOCATION_COUNTRY', 'LAST_UPDATED', 'SUMMARY', 'COMPANY_WEBSITE', 'COMPANY_LOCATION_LOCALITY', 'COMPANY_LOCATION_STREET_ADDRESS', 'COMPANY_LOCATION_CONTINENT', 'IS_PRIMARY', 'END_DATE', 'COMPANY_INDUSTRY', 'COMPANY_FACEBOOK_URL', 'COMPANY_LINKEDIN_URL', 'COMPANY_LINKEDIN_ID', 'name', 'domain', 'office_requirements', 'requirement_details', 'RTO_date_manual', 'GITHUB_URL', 'LINKEDIN_URL', 'LAST_INITIAL', 'BIRTH_DATE', 'LOCATION_ADDRESS_LINE_2', 'LOCATION_GEO', 'GENDER', 'LAST_NAME', 'TWITTER_URL', 'LOCATION_NAME', 'PERSONAL_EMAILS', 'LOCATION_COUNTRY', 'JOB_LAST_UPDATED', 'DATASET_RELEASE_DATE', 'FIRST_NAME', 'BIRTH_YE

In [24]:
final_df.write.mode("overwrite").parquet(data_dir + "people_tenure_full.parquet")



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Calculate education

In [25]:
final_df = spark.read.parquet(data_dir + "people_tenure_full.parquet")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# merge in education TODO: recalculate
# note a person can have multiple degrees so there's ID duplicates
edu = spark.read.option("multiLine", "true") \
                .option("escape", "\"") \
                .option("quote", '"').csv(data_ppl + "people-edu/", header=True)
# edu = edu.filter(F.col("SCHOOL_TYPE") == "post-secondary institution")
# start and end date are missing less often than degrees, so calculate years of schooling
edu = edu.withColumn("edu_days", (F.col("SCHOOL_TYPE") ==  "post-secondary institution").cast(IntegerType()) * F.datediff(F.col("END_DATE"), F.col("START_DATE")))
edu = edu.groupBy("ID").agg(F.sum("edu_days").alias("edu_days"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# fill in 0 for those without post-secondary 
edu = edu.withColumn("edu_days", F.when(F.col("edu_days").isNull(), 0).otherwise(F.col("edu_days")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
final_df = final_df.join(edu, on = "ID", how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
final_df.write.mode("overwrite").parquet(data_dir + "people_tenure_full_edu.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Prep for disco estimation 

In [30]:
final_df = spark.read.parquet(data_dir + "people_tenure_full_edu.parquet")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
final_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID', 'COMPANY_NAME', 'COMPANY_RAW', 'COMPANY_ID', 'COMPANY_FOUNDED', 'COMPANY_TWITTER_URL', 'COMPANY_LOCATION_POSTAL_CODE', 'COMPANY_LOCATION_REGION', 'COMPANY_LOCATION_GEO', 'TITLE_NAME', 'TITLE_LEVELS', 'COMPANY_SIZE', 'COMPANY_LOCATION_METRO', 'COMPANY_LOCATION_ADDRESS_LINE_2', 'TITLE_ROLE', 'TITLE_SUB_ROLE', 'COMPANY_LOCATION_NAME', 'START_DATE', 'TITLE_RAW', 'COMPANY_LOCATION_COUNTRY', 'LAST_UPDATED', 'SUMMARY', 'COMPANY_WEBSITE', 'COMPANY_LOCATION_LOCALITY', 'COMPANY_LOCATION_STREET_ADDRESS', 'COMPANY_LOCATION_CONTINENT', 'IS_PRIMARY', 'END_DATE', 'COMPANY_INDUSTRY', 'COMPANY_FACEBOOK_URL', 'COMPANY_LINKEDIN_URL', 'COMPANY_LINKEDIN_ID', 'name', 'domain', 'office_requirements', 'requirement_details', 'RTO_date_manual', 'GITHUB_URL', 'LINKEDIN_URL', 'LAST_INITIAL', 'BIRTH_DATE', 'LOCATION_ADDRESS_LINE_2', 'LOCATION_GEO', 'GENDER', 'LAST_NAME', 'TWITTER_URL', 'LOCATION_NAME', 'PERSONAL_EMAILS', 'LOCATION_COUNTRY', 'JOB_LAST_UPDATED', 'DATASET_RELEASE_DATE', 'FIRST_NAME', 'BIRTH_YE

In [32]:
final_df = final_df.withColumn("rto_ever", 
                         F.when( (F.col("office_requirements").isin(["Employee's choice", "Fully remote"]) 
                                 ), 0).otherwise(F.when(F.col("office_requirements").isNull() | (F.isnan("office_requirements")),
                                                       F.lit(None)).otherwise(1))
                        )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
test = final_df.withColumn("rto_ever", F.when((F.col("RTO_date_manual") == "never") & 
                                            (F.col("rto_ever") != 1), 0)
                               .otherwise(F.when((~F.col("RTO_date_manual").isin(["never", "missing"]) | 
                                                   (F.col("RTO_date_manual").isNotNull())) & 
                                                  ((F.col("rto_ever") != 0) | (F.col("rto_ever").isNull())), 1)
                                                 .otherwise(F.col("rto_ever"))))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
fn = data_ppl + "company-data-and-insights/data_0_0_10.csv.gz"
temp = pd.read_csv(fn)
schem = spark.createDataFrame(temp)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# Adjust the read statement to handle multi-line fields correctly
comp = spark.read.option("header", "true") \
               .option("multiLine", "true") \
                .option("escape", "\"") \
                .option("quote", '"') \
               .csv(data_ppl + "company-data-and-insights/", schema=schem.schema)

comp = comp.dropDuplicates() # there are a bunch of identical duplicates

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# I checked and this is a one-to-one match
test = test.join(comp.select("ID", "NAICS").withColumnRenamed("ID", "COMPANY_ID"), on = "COMPANY_ID", how="left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
test = test.filter( ((F.col("RTO_date_manual") != "missing") & (F.col("RTO_date_manual").isNotNull())) | 
                   (F.col("rto_ever") ==  0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
test = test.withColumn("tenure", F.col("tenure").cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:

df_with_naics = test.withColumn("naics_code", F.regexp_extract("NAICS", '\\\\"naics_code\\\\":\\\\"(\\d+)', 1))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df_with_naics.select("ID", "tenure", "time_col", "id_col", "month", "COMPANY_NAME", "RTO_date_manual", "edu_days", "title_categorical", "START_DATE", "END_DATE", 
           "naics_code").write.mode("overwrite").parquet(data_dir + "people_tenure_prepped")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df = spark.read.parquet(data_dir + "people_tenure_prepped")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID', 'tenure', 'time_col', 'id_col', 'month', 'COMPANY_NAME', 'RTO_date_manual', 'edu_days', 'title_categorical', 'START_DATE', 'END_DATE', 'naics_code']

In [4]:
data_dir

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://ipsos-dvd/ppl/data/'